In [5]:
!python -V

Python 3.9.7


In [121]:
import pickle
from pathlib import Path

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.neighbors import KNeighborsRegressor

from sklearn.metrics import mean_squared_error

import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [29]:
data_folder = str(Path.home())+'/mlops-zoomcamp-my-code/week1-intro-environment/notebooks/data/'

In [30]:
categorial = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [36]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
    
    df = df[(df.duration >= 1) & (df.duration <= 60)]
    
    categorical = ['PULocationID', 'DOLocationID']
    df[categorial] = df[categorical].astype(str)
    
    return df

In [38]:
df_train = read_dataframe(data_folder+'/green_tripdata_2021-01.parquet')
df_val = read_dataframe(data_folder+'/green_tripdata_2021-02.parquet')

In [39]:
len(df_train), len(df_val)

(73908, 61921)

In [40]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [41]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [42]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

10.499110706512404

In [43]:
lr = Lasso()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

12.212583224318818

In [44]:
lr = Lasso(alpha=0.001)
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

10.481255832596219

In [45]:
lr = Ridge()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

10.860753390357996

In [46]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [47]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [48]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.7587151974993125

In [50]:
with mlflow.start_run():
    
    mlflow.set_tag("developer", "ferdinand")

    mlflow.log_param("train_data-path", data_folder+'/green_tripdata_2021-01.parquet') 
    mlflow.log_param("val_data-path", data_folder+'/green_tripdata_2021-02.parquet') 
    
    alpha = 0.01
    mlflow.log_param("alpha", alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric("rmse", rmse)

In [39]:
lr = Ridge()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

11.342603943250333

In [40]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.758715202218265

In [41]:
with open ('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

## Hyperopt

In [51]:
import xgboost as xgb

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [59]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [60]:
def objective(params):
    
    with mlflow.start_run():
        mlflow.set_tag('model', 'xgboost')
        mlflow.log_params(params)
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50,
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric('rmse', rmse)
    
    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0), # [exp(-3), exp(0)] = [0.05, 1]
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42,
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

In [69]:
mlflow.xgboost.autolog(disable=True)

In [73]:
# Assuming we found the best parameter combination:
with mlflow.start_run():

    best_params = {
        'learning_rate': 0.09585355369315604,
        'max_depth': 30,
        'min_child_weight': 1.060597050922164,
        'objective': 'reg:linear',
        'reg_alpha': 0.018060244040060163,
        'reg_lambda': 0.011658731377413597,
        'seed': 42,
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=1000,
        evals=[(valid, 'validation')],
        early_stopping_rounds=50
    )

    y_pred = booster.predict(valid)
    rmse = mean_squared_error(y_val, y_pred, squared=False)
    mlflow.log_metric('rmse', rmse)

    # To save the preprocessor (the DictVectorizer)
    with open('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact('models/preprocessor.b', artifact_path='preprocessor')

    mlflow.xgboost.log_model(booster, artifact_path="models_mlflow")

[15:26:45] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation-rmse:19.48425
[1]	validation-rmse:17.95634
[2]	validation-rmse:16.59114
[3]	validation-rmse:15.37412
[4]	validation-rmse:14.29011
[5]	validation-rmse:13.32800
[6]	validation-rmse:12.47570
[7]	validation-rmse:11.72140
[8]	validation-rmse:11.05888
[9]	validation-rmse:10.47583
[10]	validation-rmse:9.96483
[11]	validation-rmse:9.51784
[12]	validation-rmse:9.12620
[13]	validation-rmse:8.78522
[14]	validation-rmse:8.49037
[15]	validation-rmse:8.23332
[16]	validation-rmse:8.01179
[17]	validation-rmse:7.81822
[18]	validation-rmse:7.65209
[19]	validation-rmse:7.50743
[20]	validation-rmse:7.38235
[21]	validation-rmse:7.27382
[22]	validation-rmse:7.17865
[23]	validation-rmse:7.09606
[24]	validation-rmse:7.02481
[25]	validation-rmse:6.96336
[26]	validation-rmse:6.91001
[27]	validation-rmse:6.86273
[28]	validation-rmse:6.82111
[29]	validation-rmse:6.78450
[30]	valida

### Loading the logged model

There are two ways to load the logged model,
* either using the `mlflow.pyfunc` module,
* or using the API for xgboost.

In [75]:
# Possibility 1:
logged_model = 'runs:/c1e4375288114b19b25a1981c3cc4f54/models_mlflow'

# Load model as a PyFuncModel:
loaded_model = mlflow.pyfunc.load_model(logged_model)

loaded_model

[15:33:25] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


mlflow.pyfunc.loaded_model:
  artifact_path: models_mlflow
  flavor: mlflow.xgboost
  run_id: c1e4375288114b19b25a1981c3cc4f54

In [76]:
# Possibility 2: 
# Using the API for xgboost
xgboost_model = mlflow.xgboost.load_model(logged_model)

xgboost_model

[15:34:43] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [77]:
# Use the loaded model:
y_pred = xgboost_model.predict(valid)
y_pred[:10]

array([14.940359 ,  7.1591063, 14.152481 , 24.3301   ,  9.4101   ,
       17.095747 , 11.566424 ,  8.471772 ,  8.940067 , 20.337719 ],
      dtype=float32)

## Trying two other models

First: Random Forest

In [119]:
def objective_rf(params):

    with mlflow.start_run():
        mlflow.set_tag('model', 'RandomForestRegressor')
        mlflow.log_params(params)
        est = int(params['n_estimators'])
        md = int(params['max_depth'])
        msl = int(params['min_samples_leaf'])
        mss = int(params['min_samples_split'])
        seed = int(params['seed'])
        model = RandomForestRegressor(
            n_estimators=est,
            max_depth=md,
            min_samples_leaf=msl,
            min_samples_split=mss,
            random_state=seed
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric('rmse', rmse)

        with open('models/preprocessor.b', 'wb') as f_out:
            pickle.dump(dv, f_out)

        mlflow.sklearn.log_model(model, artifact_path='models_mlflow')
    
    return {'loss': rmse, 'status': STATUS_OK}

In [116]:
search_space = {
    'n_estimators': hp.quniform('n_estimators', 80, 120, 1),
    'max_depth': hp.quniform('max_depth', 20, 21, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 2, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 3, 1),
    'seed': 42
}

best_result = fmin(
    fn = objective_rf, 
    space = search_space,
    algo=tpe.suggest,
    max_evals=3,
    trials=Trials()
)



100%|██████████| 3/3 [03:16<00:00, 65.39s/trial, best loss: 6.623323502237649]


Second: LightGBM

In [122]:
def objective_knn(params):

    with mlflow.start_run():
        mlflow.set_tag('model', 'KNearestNeighbor')
        mlflow.log_params(params)
        nei = int(params['n_neighbors'])
        ls = int(params['leaf_size'])
        model = KNeighborsRegressor(
            n_neighbors=nei,
            leaf_size=ls
        )
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric('rmse', rmse)

        with open('models/preprocessor.b', 'wb') as f_out:
            pickle.dump(dv, f_out)
        
        mlflow.sklearn.log_model(model, artifact_path='models_mlflow')
    
    return {'loss': rmse, 'status': STATUS_OK}

In [123]:
search_space = {
    'n_neighbors': hp.quniform('n_neighbors', 3, 10, 1),
    'leaf_size': hp.quniform('leaf_size', 20, 40, 1),
}

best_result = fmin(
    fn = objective_knn, 
    space = search_space,
    algo=tpe.suggest,
    max_evals=3,
    trials=Trials()
)

100%|██████████| 3/3 [06:28<00:00, 129.61s/trial, best loss: 6.471605600876469]
